In [1]:
import torch.nn.functional as F
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def test(model, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output, *_ = model(data)
            # get the index of the max log-probability
            pred = output.argmax(dim=1, keepdim=True)
            _, idx = output.max(dim=1)
            correct += (idx == target).sum().item()

    print('Test set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [2]:
from torch.utils import data
from torchvision import datasets
from torchvision import transforms

train_loader = data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=256, shuffle=True, drop_last=True)

test_loader = data.DataLoader(
    datasets.MNIST('./data', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=2048, shuffle=False, drop_last=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 86467808.83it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 89070363.11it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 44317580.54it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 34325277.06it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
for batch_idx, (data, target) in enumerate(train_loader):
    break

In [15]:
X = data.numpy()
X[0, :, :, :].reshape(-1).squeeze().shape

(784,)

In [3]:
import torch.nn as nn
from torch.optim import Adam


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, 5, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.MaxPool2d(2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, 5, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.MaxPool2d(2)
        )

        self.fc1 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ReLU(True)
        )
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        conv1 = self.conv1(x)
        conv2 = self.conv2(conv1)
        flatten = conv2.view(x.shape[0], -1)
        fc1 = self.fc1(flatten)
        fc2 = self.fc2(fc1)
        return fc2, [conv1, conv2]

In [4]:
import numpy as np
from deltas.classifiers.large_margin_loss import LargeMarginLoss

lm = LargeMarginLoss(
    gamma=10000,
    alpha_factor=4,
    top_k=1,
    dist_norm=np.inf
)


def train_lm(model, train_loader, optimizer, epoch, lm):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        one_hot = torch.zeros(len(target), 10).scatter_(
            1, target.unsqueeze(1), 1.).float()
        one_hot = one_hot.cuda()
        optimizer.zero_grad()
        output, feature_maps = model(data)
        # loss = F.mse_loss(output, target) * 5e-4 # l2_loss_weght
        loss = lm(output, one_hot, feature_maps)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


net = Net().to(device)
# net = nn.DataParallel(net).to(device)
optim = Adam(net.parameters())
for i in range(0, 5):
    train_lm(net, train_loader, optim, i, lm)
    test(net, test_loader)

Train Epoch: 0 [0/60000 (0%)]	Loss: 0.028109
Train Epoch: 0 [25600/60000 (43%)]	Loss: -1754.857300
Train Epoch: 0 [51200/60000 (85%)]	Loss: -2689.001465
Test set: Accuracy: 6913/10000 (69%)

Train Epoch: 1 [0/60000 (0%)]	Loss: -2679.130127
Train Epoch: 1 [25600/60000 (43%)]	Loss: -2890.081543
Train Epoch: 1 [51200/60000 (85%)]	Loss: -2983.936035
Test set: Accuracy: 6967/10000 (70%)

Train Epoch: 2 [0/60000 (0%)]	Loss: -3150.070312
Train Epoch: 2 [25600/60000 (43%)]	Loss: -2939.409180
Train Epoch: 2 [51200/60000 (85%)]	Loss: -2914.127686
Test set: Accuracy: 7912/10000 (79%)

Train Epoch: 3 [0/60000 (0%)]	Loss: -3191.222168
Train Epoch: 3 [25600/60000 (43%)]	Loss: -4553.310547
Train Epoch: 3 [51200/60000 (85%)]	Loss: -4522.619141
Test set: Accuracy: 9877/10000 (99%)

Train Epoch: 4 [0/60000 (0%)]	Loss: -4689.062988
Train Epoch: 4 [25600/60000 (43%)]	Loss: -4668.726562
Train Epoch: 4 [51200/60000 (85%)]	Loss: -4724.910156
Test set: Accuracy: 9898/10000 (99%)

